# Chapter 6: Reinforcement Learning#

Concept of learning by doing with rewards and punishment. The Machine uses it to predict a future state at t+1 with all the data available up till t.

5 different advertisements: which one is the best one? Each has a different distribution of customer spendings. We want to find the best one while we are exploring so as to limit the amount of money not invested in the best one. <br>
The regular exploration way would be a big AB-test. This type of problem is known as a **Multi-arm bandit Problem**.

Description:
+ We have d ads, each time a customer enters the website a random ad is displayed
+ If he clicks on it, reward is 1 if not reward is 0
+ Goal is to maximize the reward

## 1. Upper Confidence Bound ##

The first way of tackling such a problem is the UCB.

Process is:
+ Assign confidence band for each machine expected return
+ Pick the one with the highest Upper bound: is it a win? a loss?
+ Compute the average return and compute the new Upper and Lower bounds

Coding process is:
+ **Step 1**: For each ad {i}, compute:
    <br>$N_i(n)$ i.e. the number of times the ad was selected up till round n. 
    <br>$R_i(n)$ i.e. the sum of the rewards for that specific ad up till round n.

+ **Step 2**: From these two numbers, compute:
    <br>$\bar{r_i}(n) = \frac{R_i(n)}{N_i(n)}$ i.e. the average reward for each ad up till round n.
    <br>$[\bar{r_i}(n) - \Delta_i(n) ; \bar{r_i}(n) + \Delta_i(n)]$ i.e. the confidence interval at round n.
    <br>with $\Delta_i(n) = \sqrt{\frac{3}{2}*\frac{log(n)}{N_i(n)}}$
    
+ **Step 3**: Select the ad that has the highest $\bar{r_i}(n) + \Delta_i(n)$

In [ ]:
# Implementing UCB
import math
N = 10000
d = 10
ads_selected = []
numbers_of_selections = [0] * d
sums_of_rewards = [0] * d
total_reward = 0
for n in range(0, N):
    ad = 0
    max_upper_bound = 0
    for i in range(0, d):
        if (numbers_of_selections[i] > 0):
            average_reward = sums_of_rewards[i] / numbers_of_selections[i]
            delta_i = math.sqrt(3/2 * math.log(n + 1) / numbers_of_selections[i])
            upper_bound = average_reward + delta_i
        else:
            upper_bound = 1e400
        if upper_bound > max_upper_bound:
            max_upper_bound = upper_bound
            ad = i
    ads_selected.append(ad)
    numbers_of_selections[ad] = numbers_of_selections[ad] + 1
    reward = dataset.values[n, ad]
    sums_of_rewards[ad] = sums_of_rewards[ad] + reward
    total_reward = total_reward + reward

# Visualising the results
plt.hist(ads_selected)
plt.title('Histogram of ads selections')
plt.xlabel('Ads')
plt.ylabel('Number of times each ad was selected')
plt.show()

## 2. Thomson Sampling ##

Other way: we try to guess where the expected return of each machine is. We are not trying to guess the distribution behind machines.

Process is:
+ Pick each machine once
+ Imagine the possible distribution for each machine
+ Do an imaginary pick of each machine, actually pick the one that gives the best imaginary run
+ Refine the distribution of that machine
+ Repeat, you'll naturally refine more and more the best machine until you get good enough distribution that will let you stay with the perfect machine.

| UCB | Thomson Sampling |
|:-----: | :-----: |
|Deterministic|Probabilistic|
|Requires update every round|Can accomodate delayed feedback|
||Better Empirical Evidence|


Coding process is:
+ **Step 1**: At each round, we need to compute for each ad {i}:
    <br>$N_i^1(n)$ i.e. the number of times the ad i had reward 1 up till round n. 
    <br>$N_i^0(n)$ i.e. the number of times the ad i had reward 0 up till round n.

+ **Step 2**: For each ad, we take a random draw from the distribution below:
    <br>$\theta_i(n) = \beta(N_i^1(n) + 1 ; N_i^0(n) + 1)$
    
+ **Step 3**: Select the ad that has the highest $\theta_i(n)$